# Robo Trade 2.0 Usando Deep Q-Learning

Segunda tentativa de criação de uma rede neural para prever oscilações no mercado de ação.

Desta vez tentaremos usar Deep Q-Learning, usando Q-Networks.

<hr>

## Funcionamento

Nossa rede neural terá a seguinte configuração:

* **Entrada:** os últimos X estados escolhidos pela variável ```window_size```.
* **Saídas:** Comprar, Vender, Esperar.

<hr>

## Classes e suas funções
### **AI_Trader:**
- **Construtor:**

  Inicializa o agente que atuará no nosso ambiente.

- **Model Builder:**

  Modela a arquitetura da nossa rede neural de acordo com nossas escolhas.

- **Trade:**

  Função de decide se o agente irá executar um previsão usando a Q-Network ou executará uma ação gananciosa aleatória.

  Esta também é a função que retorna a resposta final da rede neural (Comprar, Vender ou Esperar).

- **Batch_Trade:**

  Função que realiza o treinamendo do lote de memórias.





In [ ]:
# EXECUTAR NA PRIMEIRA EXECUÇÃO!

In [ ]:
pip install yfinance

     |████████████████████████████████| 63 kB 1.4 MB/s 
     |████████████████████████████████| 6.4 MB 11.2 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
# EXECUTAR NA PRIMEIRA EXECUÇÃO!

# Bitcoin Hora a hora - Data download
!gdown --id '1VQry5JMRcuZ_BStIX8-FB8n4zFuDNUjk'

In [ ]:
# Imports

import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader

from tqdm import tqdm_notebook, tqdm
from collections import deque

import pandas
from pandas_datareader import data as pdr
import yfinance as yfin
import datetime

import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

debug = 1

In [ ]:
# Defining our Deep Q-Learning Trader

class AI_Trader():  

# -----------------------------------------------------------------------

  # CONSTRUTOR

  def __init__(self, state_size, action_space=3, model_name="AITrader"):
    
    self.state_size = state_size # Tamanho da entrada da rede neural 
    self.action_space = action_space # Espaço de ação será 3, Comprar, Vender, Sem Ação (Tamanho da saída da rede neural)
    self.memory = deque(maxlen=2000) # Memória com 2000 posições. A função Deque permite adicionar elementos ao final, enquanto remove elementos do início.
    self.inventory = [] # Terá as comprar que já fizemos
    self.model_name = model_name # Nome do modelo para o Keras
    
    self.gamma = 0.95 # Parâmetro que ajudará a maximizar a recompensa
    self.epsilon = 1.0 # Taxa de aleatoriedade para atitudes ganacioas do algorítimo.
    self.epsilon_final = 0.01 # Taxa final reduzida
    self.epsilon_decay = 0.995 # Velocidade de decaimento da taxa

    self.model = self.model_builder() # Inicializa um modelo e de rede neural e salva na classe

# -----------------------------------------------------------------------

  # DEFININDO A REDE NEURAL

  def model_builder(self):
        
    model = tf.keras.models.Sequential()      
    model.add(layers.Dense(units=32, activation='relu', input_dim=self.state_size))
    model.add(layers.Dense(units=64, activation='relu'))
    model.add(layers.Dense(units=128, activation='relu'))
    model.add(layers.Dense(units=self.action_space, activation='linear')) # De maneira geral, teremos 3 saída na rede geral (número de espaços de ação)


    model.compile(loss='mse', optimizer=keras.optimizers.Adam(learning_rate=0.001)); # Compilamos o modelo

    return model # Retornamos o modelo pela função.

# -----------------------------------------------------------------------

  # FUNÇÃO DE TRADE
  # Usa o Epsilon e um número aleatório para definir se usará um dado aleatório ou a previsão da rede.

  def trade(self, state):
    if(debug):{print('TRADE FUNCTION:')}

    if random.random() <= self.epsilon:
      if(debug):{print('Entrou - Random')}
      return random.randrange(self.action_space)

    if(debug):{print('Vai Treinar Modelo')}
    actions = self.model.predict(state)
    if(debug):{print('Actions = ', actions)}
    if(debug):{print('Actions Argmax = ', np.argmax(actions[0]))}

    return np.argmax(actions[0])

# -----------------------------------------------------------------------

  # LOTE DE TREINAMENTO

  # Definindo o modelo para treinamento do lote

  def batch_train(self, batch_size): # Função que tem o tamanho do lote como argumento

    batch = [] # Iremos usar a memória como lote, por isso iniciamos com uma lista vazia

    # Iteramos sobre a memória, adicionando seus elementos ao lote batch
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)): 
      batch.append(self.memory[i])

    # Agora temos um lote de dados e devemos iterar sobre cada estado, recompensa,
    # proximo_estado e conclusão do lote e treinar o modelo com isso.
    for state, action, reward, next_state, done in batch:
      reward = reward

      # Se não estivermos no último agente da memória, então calculamos a
      # recompensa descontando a recompensa total da recompensa atual.
      if not done:
        reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])

      # Fazemos uma previsão e alocamos à varivel target
      target = self.model.predict(state)
      target[0][action] = reward

      # Treinamos o modelo com o estado, usando a previsão como resultado esperado.
      self.model.fit(state, target, epochs=1, verbose=0)

    # Por fim decrementamos o epsilon a fim de gradativamente diminuir tentativas ganaciosas. 
    if self.epsilon > self.epsilon_final:
      self.epsilon *= self.epsilon_decay

# -----------------------------------------------------------------------


# -----------------------------------------------------------------------


# -----------------------------------------------------------------------


# -----------------------------------------------------------------------
    


In [ ]:
# Stock Market Data Preprocessing

# Definiremos algumas funções uteis

# Sigmoid
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

# Função para formatar texto
def stock_price_format(n):
  if n < 0:
    return "- # {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))

# Busca dados no Yahoo Finance
# Formato data = "yyyy-mm-dd"
def dataset_loader(stock_name, initial_date, final_date):

  yfin.pdr_override()

  dataset = pdr.get_data_yahoo(stock_name, start=initial_date, end=final_date)
  
  start_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[1]).split()[0]
  
  close = dataset['Close']
  
  return close

# State Creator

Primeiro vamos traduzir o problema para um ambiente de aprendizado por reforço.

* Cada ponto no gráfico é um ponto flutuante que representa o valor no momento do tempo.

* Devemos prever o que acontecerá no próximo período de tempo, usando umas das 3 possibilidades de ação: compra, venda ou sem ação (esperar)

Inicialmente vamos usar uma janela de 5 estados anteriores, para tentar prever o próximo.

```windows_size = 5```

Ao invés vez de prever valores reais para nosso alvo, queremos prever uma de nossas 3 ações.

Em seguida, mudamos nossos estados de entrada para diferenças nos preços das ações, que representarão as mudanças de preços ao longo do tempo.


In [ ]:
# State Creator


def state_creator(data, timestep, window_size):

  # O index incial (starting_id) será o timestep (passos/dias que já foram dados)
  # menos o tamanho da janela, que serão os dias olhados para trás.
  starting_id = timestep - window_size + 1

  if(debug):{print('Timestep = ', timestep)}
  if(debug):{print('Window_size = ', window_size)}
  if(debug):{print("Starting id = ", starting_id)}

  if starting_id >= 0:
    windowed_data = data[starting_id: timestep + 1]

    if(debug):{print("Entrou no >=0. Starting id = ", starting_id)}
    if(debug):{print("windowed_data = ", windowed_data)}

  else:
    windowed_data =- starting_id * [data[0]] + list(data[0:timestep + 1])

    if(debug):{print("Entrou no Else. Starting id = ", starting_id)}
    if(debug):{print("w_d = ", windowed_data)}

  state = [] # Criou uma array vazia para o estado

  if(debug):{print('Vai entrar no FOR de normalização:')}

  for i in range(window_size - 1):
    if(debug):{print('windowed_data[i + 1] = ', windowed_data[i+1])}
    if(debug):{print('windowed_data[i] = ', windowed_data[i])}

    state.append(sigmoid(windowed_data[i + 1] - windowed_data[i]))

    if(debug):{print('state = ',state)}

  return np.array([state])

In [ ]:
# Loading a Dataset

# CONFIGURAÇÕES DE IMPORTAÇÃO DE DADOS

# NOME DA AÇÃO
STOCK_NAME = "WEGE3.SA"

# DATA INCIAL
INITIAL_DATE = "2021-01-01"

# DATA FINAL
today = datetime.date.today()
FINAL_DATE = today.strftime("%Y-%m-%d") # Escolhe a data final como hoje

data = dataset_loader(STOCK_NAME, INITIAL_DATE, FINAL_DATE);

data

[*********************100%***********************]  1 of 1 completed


Date
2021-01-04    37.310001
2021-01-05    39.599998
2021-01-06    40.650002
2021-01-07    42.330002
2021-01-08    44.889999
                ...    
2022-02-14    30.440001
2022-02-15    32.880001
2022-02-16    31.299999
2022-02-17    30.650000
2022-02-18    29.809999
Name: Close, Length: 282, dtype: float64

In [ ]:
# Training the Q-Learning Trading Agent

window_size = 10
episodes = 2

batch_size = 32
data_samples = len(data) - 1

trader = AI_Trader(window_size)
trader.model.summary()

# if(debug):{print('Entrou - Random')}

trader.epsilon = 1


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 32)                352       
                                                                 
 dense_5 (Dense)             (None, 64)                2112      
                                                                 
 dense_6 (Dense)             (None, 128)               8320      
                                                                 
 dense_7 (Dense)             (None, 3)                 387       
                                                                 
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


In [ ]:
debug = 1

state = state_creator(data, 0, window_size + 1)

total_profit = 0;
trader.inventory = []

t=14



In [ ]:
trader.epsilon = 0.1


if(debug):{print('------ MAKE TRADE:')}

action = trader.trade(state)

if(debug):{print('ACTION = ', action)}

if(debug):{print('------ FIND NEXT STATE:')}

next_state = state_creator(data, t+1, window_size + 1)
reward = 0

# action = 1



if(debug):{print('------ TAKING ACTION:')}
if(debug):{print('Tader Inventory = ', trader.inventory)}


if action == 1: #Buying
  if(debug):{print('ACTION = 1 (BUY)')}

  trader.inventory.append(data[t])
  print("AI Trader bought: ", stock_price_format(data[t]))

elif action == 2 and len(trader.inventory) > 0: #Selling
  if(debug):{print('ACTION = 0 (SELL)')}

  buy_price = trader.inventory.pop(0)
  if(debug):{print('Buy Price = ', buy_price)}
  
  reward = max(data[t] - buy_price, 0)
  if(debug):{print('dat[t] = ', data[t])}
  if(debug):{print('Reward = ', reward)}

  total_profit += data[t] - buy_price
  if(debug):{print('Total Pofit = ', total_profit)}
  print("AI Trader sold: ", stock_price_format(data[t]), " Profit: " + stock_price_format(data[t] - buy_price) )


if t == data_samples - 1:
    done = True
else:
    done = False

if(debug):{print('------ SAVING MEMORY:')}

trader.memory.append((state, action, reward, next_state, done))
if(debug):{print('Memory = ', trader.memory)}

    
state = next_state

t = t+1

# Se o tamanho da memória for maior que o tamanho do lote que definimos
# Então vamos treinar a rede, passando o tamanho do lote como argumento
if(debug):{print('Memory Len= ', len(trader.memory))}

if len(trader.memory) > batch_size:
  trader.batch_train(batch_size)

------ MAKE TRADE:
TRADE FUNCTION:
Vai Treinar Modelo
Actions =  [[0.02785273 0.19441126 0.09472802]]
Actions Argmax =  1
ACTION =  1
------ FIND NEXT STATE:
Timestep =  25
Window_size =  11
Starting id =  15
Entrou no >=0. Starting id =  15
windowed_data =  Date
2021-01-26    44.474998
2021-01-27    44.345001
2021-01-28    43.985001
2021-01-29    41.895000
2021-02-01    42.270000
2021-02-02    42.615002
2021-02-03    44.334999
2021-02-04    42.785000
2021-02-05    42.250000
2021-02-08    42.750000
2021-02-09    42.505001
Name: Close, dtype: float64
Vai entrar no FOR de normalização:
windowed_data[i + 1] =  44.345001220703125
windowed_data[i] =  44.474998474121094
state =  [0.4675463773653247]
windowed_data[i + 1] =  43.98500061035156
windowed_data[i] =  44.345001220703125
state =  [0.4675463773653247, 0.4109594181924409]
windowed_data[i + 1] =  41.89500045776367
windowed_data[i] =  43.98500061035156
state =  [0.4675463773653247, 0.4109594181924409, 0.11007255941551251]
windowed_data[i

In [ ]:
trader.memory

In [ ]:
a = [1,2,100,4,5,6,7]

print(np.argmax(a))

2


In [ ]:
# Defining a Training Loop

# Vamos iterar sobre todos episódios



for episode in range(1, episodes + 1):
  
  print("Episode: {}/{}".format(episode, episodes))
  
  state = state_creator(data, 0, window_size + 1)
  
  total_profit = 0
  trader.inventory = []
  
  for t in tqdm(range(data_samples)):
    
    action = trader.trade(state)
    
    next_state = state_creator(data, t+1, window_size + 1)
    reward = 0
    
    if action == 1: #Buying
      trader.inventory.append(data[t])
      print("AI Trader bought: ", stock_price_format(data[t]))
      
    elif action == 2 and len(trader.inventory) > 0: #Selling
      buy_price = trader.inventory.pop(0)
      
      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price
      print("AI Trader sold: ", stock_price_format(data[t]), " Profit: " + stock_price_format(data[t] - buy_price) )
      
    if t == data_samples - 1:
      done = True
    else:
      done = False
      
    trader.memory.append((state, action, reward, next_state, done))
    
    state = next_state
    
    if done:
      print("########################")
      print("TOTAL PROFIT: {}".format(total_profit))
      print("########################")
    
    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)
      
  if episode % 10 == 0:
    trader.model.save("ai_trader_{}.h5".format(episode))
    